# Sampling Methods

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Simple_random_sampling.PNG" width="400px" height="400px" />

> We now have a suite of methods for **exact** inference. Although these methods exploit the graphical structure of our networks, they are not efficient (in the general case) in the sense that the time they need to obtain the result is exponential in the size of the problem.
>
> Therefore, we will study sampling methods to achieve approximate inference in graphical models.

> **Objetives:**
> - To remember how to estimate statistics using samples.
> - To study a simple sampling method for Bayesian networks.
> - To describe the Markov Chain Monte Carlo methods.

> **References:**
> - Probabilistic Graphical Models: Principles and Techniques, By Daphne Koller and Nir Friedman. Ch. 12.
> - Mastering Probabilistic Graphical Models Using Python, By Ankur Ankan and Abinash Panda. Ch. 4.
> - Probabilistic Graphical Models Specialization, offered through Coursera. Prof. Daphne Koller.


<p style="text-align:right;"> Imagen recuperada de: https://upload.wikimedia.org/wikipedia/commons/b/bf/Simple_random_sampling.PNG.</p>

___

# 1. Sampling in Bayesian Networks

First, let's remember how we estimate simple things using samples from a distribution $P$.

## 1.1. Simple sampling

Let $\mathcal{D}=\{x_1, x_2, \dots, x_M\}$ be a dataset of **independent and identically distributed (IID)** samples from the distribution $P(X=x)$.

For example, if $X$ is a binary random variable ($\mathrm{Val}(X)=\{x^0, x^1\}$) and $P(X=x^1)=p$, then an estimator for $p$ is:

$$\hat{p} = \frac{1}{M} \sum_{i=1}^{M} I(x_i = x^1).$$

In [ ]:
# Import numpy


In [ ]:
# np.random.choice


In [ ]:
# Sample the binary distribution


In [ ]:
# Estimation of p


More generally, for any distribution $P$ and function $f$,

$$E_P[f] \approx \frac{1}{M} \sum_{i=1}^{M}f(x_i).$$

**Application of this simple concept:**

- [Montecarlo integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration)

### How can we sample from a multinomial discrete distribution?

Let $X$ be a discrete random variable with $\mathrm{Val}(X)=\{x^1, x^2, \dots, x^k\}$, and $P(x^i) = \theta^i$.

Usually we have a pseudorandom number generator from the uniform distribution $\mathcal{U}[0, 1]$ (equal probability of drawing any number between 0 and 1):

In [ ]:
# numpy.random.rand


Thus, we can sample as shown in the following image:

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename='figures/discrete_sampling.png')

### What theoretical guarantees do we have?

> *Theorem (Hoeffding bound).* For the estimator $\hat{p} = \frac{1}{M} \sum_{i=1}^{M} I(x_i = x^1)$, the inequality
> 
> $$P_{\mathcal{D}}(\hat{p}\notin[p-\epsilon, p+\epsilon]) \leq 2 e^{-2 M \epsilon^2}$$
>
> holds.

> *Theorem (Chernoff bound).* For the estimator $\hat{p} = \frac{1}{M} \sum_{i=1}^{M} I(x_i = x^1)$, the inequality
> 
> $$P_{\mathcal{D}}(\hat{p}\notin[p(1-\epsilon), p(1+\epsilon)]) \leq 2 e^{- M p \epsilon^2 / 3}$$
>
> holds.

**Comments:**

1. Hoeffding: additive error
2. Chernoff: multiplicative errror
3. Both say that "the probability of a bad sample set (with error $\epsilon$)" is a decreasing function of the number of samples.
4. These guarantees are extended for estimation over general functions.
5. Issues appear when $p$ (the true value is small): $\epsilon$ needs to be small relative to $p$.

**Exercise.**

Assume that we want to be at least $1-\delta$ confident that we will have a good estimator. How many samples do we need?

## 1.2. Forward sampling in Bayesian networks

There are several ways to perform sampling over a Bayesian network. The *forward sampling* is perhaps the easiest and practical way to do it, since it is done in the natural calusal direction of the model.

Let's consider the student example:

In [ ]:
Image(filename='figures/Student1.png')

The sampling is done in the topological order:

1. We begin with the nodes that don't have parents ($D,I$):
   - $d^0, i^1$.

2. Once the parents of a node have been sampled, we can sample that node using the corresponding row of the conditional distribution ($C,E$ using the rows corresponding to $i^1, d^0$):
   - $c^0, e^0$.
   - $r^1$

3. Finally, the complete sample is $d^0, i^1, c^0, e^0, r^1$.

### How do we estimate probabilities using these samples?

1. Goal: To estimate $P(\bar{Y}=\bar{y})$ (or a function of $\bar{Y}$).

   - Generate and collect samples from the Bayesian network. How many? 
   - Count the number of times that $\bar{Y}=\bar{y}$ and compute its relative frequency.

### What if we have evidence?

2. Goal: To estimate $P(\bar{Y}=\bar{y} | \bar{E}=\bar{e})$.

   - Rejection sampling algorithm:
     - Generate samples from the Bayesian network.
     - Throw away all those where $\bar{E} \neq \bar{e}$
     - Count the number of times that $\bar{Y}=\bar{y}$ and compute its relative frequency.
     
   - Expected fraction of samples kept after throwing away those that are not consistent with the evidence: $P(\bar{e})$.
   - Thus, we should generate: $M\geq \frac{\log(2/\delta)}{2P(\bar{e})\epsilon^2}$ samples

> The number of samples grows exponentially with the number of observed variables.

### How is this done with `pgmpy`?

First, let's compute the exact inference:

In [ ]:
# Import pgmpy.factors.discrete.TabularCPD

# Import pgmpy.models.BayesianModel


In [ ]:
# Defining the model structure. We can define the network by just passing a list of edges.


In [ ]:
# Defining individual CPDs


In [ ]:
# Associating the CPDs with the network


Compute the marginal probability $P(C)$

In [ ]:
# Import pgmpy.inference.VariableElimination


Now, let's see what we get using forward sampling:

In [ ]:
# Import pgmpy.sampling.BayesianModelSampling


In [ ]:
# Instantiate a sampling object


In [ ]:
# Number of samples for error of 1% and confidence of 99%


In [ ]:
# Generate samples


In [ ]:
# Estimation of P(C)


In [ ]:
# Compare


Unfortunately, this method cannot be extended to Markov networks.

# 2. Markov Chain Monte Carlo

The Markov Chain Monte Carlo (MCMC) methods appear as an answer to the above issue. These methods allow us to design an iterative sampling process that converges to the desired target distribution, which may be intractable to sample from directly.

Let's remember what the Markov chains are:

## 2.1. Markov chains

Recall that a **Markov chain** defines a probabilistic transition model $T(x \to x')$ over some states $x$:

$$\forall x: \sum_{x'}T(x \to x') = 1.$$

**Example:** Drunk man Markov chain

In [ ]:
Image("figures/drunkMC.png")

### How does the probability of being in a given state evolve?

Given some initial probability distribution $P^{(0)}(X^{(0)})$, we can find $P^{(t)}(X^{(t)})$ as:

$$P^{(t+1)}(X^{(t+1)} = x') = \sum_{x} P^{(t)}(X^{(t)}) T(x \to x').$$

For example, we know that the drunk man begins his journey at $X=0$. Then:

| $P^{(t)}$ | $-2$     | $-1$   | $0$     | $1$   | $2$      |
| --------- | -------- | ------ | ------- | ----- | -------- |
| $P^{(0)}$ | $0$      | $0$    | $1$     | $0$   | $0$      |
| $P^{(1)}$ | $0$      | $0.25$ | $0.5$   | $0.25$| $0$      |
| $P^{(2)}$ | $0.0625$ | $0.25$ | $0.375$ | $0.25$| $0.0625$ |

In [ ]:
# Import numpy


In [ ]:
# Transition matrix

# Initial distribution


In [ ]:
# Distribution 1, 2, ..., 10


In [ ]:
# Distribution 100


In [ ]:
# Distribution 1000


After some iterations, the distribution converges to an stationary distribution.

What happens if the initial distribution is changed?

In [ ]:
# Try different P0


In [ ]:
# Try different P0


Then, the stationary distribution satisfies:

$$\Pi(x') = \sum_{x} \Pi(x) T(x \to x'),$$

and

$$\sum_{x} \Pi(x) = 1,$$

which forms a system of linear equations.

For the above example:

In [ ]:
# Modify and augment matrix


In [ ]:
# Least squares solution


However, **not all Markov chains converge to a unique distribution regardless the initial probability distribution**.

> *Definition (Regular Markov chain).* A Markov chain is **regular** if there exists $k\in\mathbb{N}$ such that, for every $x, x' \in \mathrm{Val}(X)$, the probability of getting from $x$ to $x'$ in exactly $k$ steps is *positive* ($>0$).

> *Theorem.* A regular Markov chain converges to a unique stationary distribution regardless the initial distribution.

**Sufficient condition for regularity:**
- Every two states $x, x'$ are connected with a path of positive probability.
- For every state, there is a self-transition.

## 2.2. Using a Markov chain

Now that we know what a Markov chain is, let's examine how to use it in the context of approximate inference using sampling.

- Goal: Compute $P(x \in \mathcal{S})$.
  - But $P$ is too hard to sample from directly.
  
- Construct a Markov chain whose unique stationary distribution is $P$.

- Sample $x^{(0)}$ from $P^{(0)}$.

- For $t=0, 1, 2, \dots$:
  - Generate $x^{(t+1)}$ from $T(x^{(t)} \to x')$.

### Which samples will we use?

We only want to use samples that are sampled form a distribution which is close to $P$.

However, at early iterations, $P^{(t)}$ is usually far from $P$.

We should start collecting samples only after the chain has run long enough to **mix**.

### How do you know if a chain has mixed or not?

In general, it is very difficult to prove that a chain has mixed.

In many cases, you can prove that it has not:

 - Compare chain statistics in different windows within a single run of the chain;
 - and accross different runs initialized differently.

### Using the samples

Once the chain mixes, all samples $x^{(t)}$ are from the stationary distribution $\Pi$:

 - So we should use all $x^{(t)}$ for $t>T_{mix}$.

Nearby samples are correlated (not IID)!
 - We shouldn't overestimate the quality of our estimator by simple counting the number of samples.

### Algorithm

1. For $c=1,\dots,C$:
   - Sample $x^{(c,0)}$ from $P^{(0)}$.
   
2. Repeat until mixing:
   - For $c=1,\dots,C$:
     - Generate $x^{(c,t+1)}$ from $T(x^{(c,t)} \to x')$.
     - Compare window statistics in different chains to determine mixing.

3. Repeat until sufficient samples:
   - $\mathcal{D} = \emptyset$.
   - For $c=1,\dots,C$:
     - Generate $x^{(c,t+1)}$ from $T(x^{(c,t)} \to x')$.
     - $\mathcal{D} = \mathcal{D} \cup \{x^{(c,t+1)}\}$.

**Pros:**
- General purpose.
- Easy to implement.
- Good theoretical guarantees as $t\to\infty$.

**Cons:**
- Lots of design choices.
- Can be very slow to converge.
- Difficult to tell whether it is working.

## 2.3. Gibbs sampling

Until now we have the general framework clear. However, we still do not know how to construct a Markov chain with a desired unique stationary distribution.

One simple way to do this is using the Gibbs chain.

- Target distribution $P_{\Phi}(X_1, \dots, X_n)$.
- Markov chain state space: complete assinment $\bar{x}$ to $\bar{X} = \{X_1, \dots, X_n\}$.
- Transition model given an starting state $\bar{x}^{(0)}$:
  - For $i=1,\dots,n$:
    - Sample $x_i \sim P_{\Phi}(X_i | \bar{x}_{-i})$; where $\bar{x}_{-i}$ is an assignment to  $\bar{X} = \{X_1, \dots, X_n\}$, except for $X_i$.
  - Set $\bar{x}^{(t+1)} = \bar{x}$.

**Example:**

In [ ]:
Image("figures/Student1.png")

Let's assume that we want to sample the distribution $P_{\Phi}(D, I, C | p^1, r^0)$.

Starting sample: $d^0, i^0, c^1$.

| Sample          | Distribution to sample from   |
| --------------- | ----------------------------- |
| $d^0, i^0, c^1$ | $P(D|i^0, c^1, p^1, r^0)$     |
| $d^1, i^0, c^1$ | $P(I|d^1, c^1, p^1, r^0)$     |
| $d^1, i^1, c^1$ | $P(C|d^1, i^1, p^1, r^0)$     |
| $d^1, i^1, c^1$ | This is $x^{(1)}$             |

In [ ]:
# Import pgmpy.sampling.GibbsSampling


In [ ]:
# Instantiate a sampling object


In [ ]:
# Generate N samples


In [ ]:
# Use rolling to get statistics on windows of predefined size


In [ ]:
# Plot


In [ ]:
# Compare


<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>